<a href="https://colab.research.google.com/github/jarodchristiansen/PythonProjects/blob/master/LSTM_Music_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install youtube-dl
!pip install pydub

     |████████████████████████████████| 1.8MB 2.8MB/s 


In [ ]:
import youtube_dl

In [ ]:
url = 'https://www.youtube.com/watch?v=OqYCwK4BZK0&list=PLDE50B0B361F9E75D'

In [ ]:
ydl_opts = {
    'format': 'bestaudio/best',
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'wav',
        'preferredquality': '192',
    }],
    'outtmpl': '%(title)s.%(etx)s',
    'quiet': False
}

with youtube_dl.YoutubeDL(ydl_opts) as ydl:
    ydl.download([url])

[youtube:playlist] Downloading playlist PLDE50B0B361F9E75D - add --no-playlist to just download video OqYCwK4BZK0
[youtube:playlist] PLDE50B0B361F9E75D: Downloading webpage
[download] Downloading playlist: Blackmill Playlist
[youtube:playlist] playlist Blackmill Playlist: Downloading 36 videos
[download] Downloading video 1 of 36
[youtube] OqYCwK4BZK0: Downloading webpage
[download] Blackmill - Rain (Full Version).NA has already been downloaded
[download] 100% of 6.53MiB
[ffmpeg] Destination: Blackmill - Rain (Full Version).wav
Deleting original file Blackmill - Rain (Full Version).NA (pass -k to keep)
[download] Downloading video 2 of 36
[youtube] Sf7ubB0Ar14: Downloading webpage
[download] Destination: Blackmill - Oh Miah (Full Version).NA
[download] 100% of 5.38MiB in 00:00
[ffmpeg] Destination: Blackmill - Oh Miah (Full Version).wav
Deleting original file Blackmill - Oh Miah (Full Version).NA (pass -k to keep)
[download] Downloading video 3 of 36
[youtube] ADMkaI6YDEM: Downloading 

ERROR: bIZJuigT27Y: YouTube said: This video is no longer available due to a copyright claim by a third party.
Sorry about that.


DownloadError: ignored

In [ ]:
import numpy as np
import pandas as pd
import pydub
from keras.layers import Dense, LSTM, LeakyReLU
from keras.models import Sequential, load_model
from scipy.io.wavfile import read, write
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [ ]:
import glob
songs = glob.glob('/content/' + '/*.wav')

In [ ]:
songs

['/content/Blackmill - City Lights (Full Version).wav',
 "/content/Blackmill - As Time Goes By (ND Version) 'The Snowman Footage'.wav",
 '/content/Blackmill - A Reach For Glory (Full Version).wav',
 '/content/Blackmill - Sacred River (Full Version).wav',
 '/content/Coldplay - Trouble (Blackmill Chillstep) Free Download!.wav',
 '/content/Blackmill - Flesh and bones (Full Version).wav',
 '/content/Ellie Goulding - Your Song (Blackmill Dubstep Remix) + High Quality Download Link.wav',
 '/content/Blackmill - Rain (Full Version).wav',
 '/content/Calvin Harris - Flashback (Blackmill Dubstep Remix).wav',
 '/content/Blackmill - Oh Miah (Full Version).wav',
 '/content/Blackmill - In The Night Of Wilderness (Full Version).wav']

Converts mp3 to wav files if needed otherwise the blow code will read the wav files naturally

In [ ]:
rate, music1 = read('{}'.format(songs[0]))
rate, music2 = read('{}'.format(songs[1]))
rate, music3 = read('{}'.format(songs[2]))
rate, music4 = read('{}'.format(songs[3]))
rate, music5 = read('{}'.format(songs[4]))
rate, music6 = read('{}'.format(songs[5]))
rate, music7 = read('{}'.format(songs[6]))
rate, music8 = read('{}'.format(songs[7]))
rate, music9 = read('{}'.format(songs[8]))
rate, music10 = read('{}'.format(songs[9]))
rate, music11 = read('{}'.format(songs[10]))

In [ ]:
music1 = pd.DataFrame(music1[0:15000000, :])
music2 = pd.DataFrame(music2[0:15000000, :])
music3 = pd.DataFrame(music3[0:15000000, :])
music4 = pd.DataFrame(music4[0:15000000, :])
music5 = pd.DataFrame(music5[0:15000000, :])
music6 = pd.DataFrame(music6[0:15000000, :])
music7 = pd.DataFrame(music7[0:15000000, :])
music8 = pd.DataFrame(music8[0:15000000, :])
music9 = pd.DataFrame(music9[0:15000000, :])
music10 = pd.DataFrame(music10[0:15000000, :])
music11 = pd.DataFrame(music11[0:15000000, :])

In [ ]:
result = pd.concat([music1, music2,music3,music4,music5,music6,music7,music8,music9,music10,music11,], axis=1, join='inner')

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
# function to create training data by shifting the music data 
def create_train_dataset(df, look_back, train=True):
    dataX1, dataX2, dataY1, dataY2 = [],[],[],[]
    for i in range(len(df)-look_back-1):
        dataX1.append(df.iloc[i : i + look_back, 0].values)
        dataX2.append(df.iloc[i : i + look_back, 1].values)
        if train:
            dataY1.append(df.iloc[i + look_back, 0])
            dataY2.append(df.iloc[i + look_back, 1])
  
            
    if train:
        return np.array(dataX1), np.array(dataX2),  np.array(dataY1), np.array(dataY2)
    else:
        return np.array(dataX1), np.array(dataX2)

In [ ]:
X1, X2 = create_train_dataset(pd.concat([result[0:50000].loc[0:20000000, :],result[50000:-1].loc[0:2000000, :]], axis=0), look_back=3, train=True)

KeyboardInterrupt: ignored

In [ ]:
#test1, test2 = create_train_dataset(pd.concat([music1.iloc[1600001 : 4000000, :],music2.iloc[1600001 : 4000000, :]], axis=0), look_back=3, train=False)
test1, test2 = create_train_dataset(pd.concat([result[0:10].loc[1600001:5000000, :],result[1:2].loc[1600001:5000000, :]], axis=0), look_back=3, train=False)

In [ ]:
X1

array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]], dtype=int16)

NameError: ignored

In [ ]:
X1 = X1.reshape((-1, 1, 3))
X2 = X2.reshape((-1, 1, 3))
X3 = X3.reshape((-1, 1, 3))
X4 = X4.reshape((-1, 1, 3))
X5 = X5.reshape((-1, 1, 3))
X6 = X6.reshape((-1, 1, 3))
X7 = X7.reshape((-1, 1, 3))
X8 = X8.reshape((-1, 1, 3))
X9 = X9.reshape((-1, 1, 3))
X10 = X10.reshape((-1, 1, 3))
X11 = X11.reshape((-1, 1, 3))
test1 = test1.reshape((-1, 1, 3))
test2 = test2.reshape((-1, 1, 3))
test3 = test3.reshape((-1, 1, 3))
test4 = test4.reshape((-1, 1, 3))
test5 = test5.reshape((-1, 1, 3))
test6 = test6.reshape((-1, 1, 3))
test7 = test7.reshape((-1, 1, 3))
test8 = test8.reshape((-1, 1, 3))
test9 = test9.reshape((-1, 1, 3))
test10 = test10.reshape((-1, 1, 3))
test11 = test11.reshape((-1, 1, 3))

In [ ]:
rnn1 = Sequential()
rnn1.add(LSTM(units=100, activation='linear', input_shape=(None, 3)))
rnn1.add(LeakyReLU())
rnn1.add(Dense(units=50, activation='linear'))
rnn1.add(LeakyReLU())
rnn1.add(Dense(units=25, activation='linear'))
rnn1.add(LeakyReLU())
rnn1.add(Dense(units=12, activation='linear'))
rnn1.add(LeakyReLU())
rnn1.add(Dense(units=1, activation='linear'))
rnn1.add(LeakyReLU())
rnn1.compile(optimizer='adam', loss='mean_squared_error')
rnn1.fit(X1, y1, epochs=20, batch_size=100)

In [ ]:
rnn2 = Sequential()
rnn2.add(LSTM(units=100, activation='linear', input_shape=(None, 3)))
rnn2.add(LeakyReLU())
rnn2.add(Dense(units=50, activation='linear'))
rnn2.add(LeakyReLU())
rnn2.add(Dense(units=25, activation='linear'))
rnn2.add(LeakyReLU())
rnn2.add(Dense(units=12, activation='linear'))
rnn2.add(LeakyReLU())
rnn2.add(Dense(units=1, activation='linear'))
rnn2.add(LeakyReLU())
rnn2.compile(optimizer='adam', loss='mean_squared_error')
rnn2.fit(X2, y2, epochs=20, batch_size=100)

In [ ]:
rnn3 = Sequential()
rnn3.add(LSTM(units=100, activation='linear', input_shape=(None, 3)))
rnn3.add(LeakyReLU())
rnn3.add(Dense(units=50, activation='linear'))
rnn3.add(LeakyReLU())
rnn3.add(Dense(units=25, activation='linear'))
rnn3.add(LeakyReLU())
rnn3.add(Dense(units=12, activation='linear'))
rnn3.add(LeakyReLU())
rnn3.add(Dense(units=1, activation='linear'))
rnn3.add(LeakyReLU())
rnn3.compile(optimizer='adam', loss='mean_squared_error')
rnn3.fit(X3, y3, epochs=20, batch_size=100)

In [ ]:
rnn4 = Sequential()
rnn4.add(LSTM(units=100, activation='linear', input_shape=(None, 3)))
rnn4.add(LeakyReLU())
rnn4.add(Dense(units=50, activation='linear'))
rnn4.add(LeakyReLU())
rnn4.add(Dense(units=25, activation='linear'))
rnn4.add(LeakyReLU())
rnn4.add(Dense(units=12, activation='linear'))
rnn4.add(LeakyReLU())
rnn4.add(Dense(units=1, activation='linear'))
rnn4.add(LeakyReLU())
rnn4.compile(optimizer='adam', loss='mean_squared_error')
rnn4.fit(X4, y4, epochs=20, batch_size=100)

In [ ]:
rnn5 = Sequential()
rnn5.add(LSTM(units=100, activation='linear', input_shape=(None, 3)))
rnn5.add(LeakyReLU())
rnn5.add(Dense(units=50, activation='linear'))
rnn5.add(LeakyReLU())
rnn5.add(Dense(units=25, activation='linear'))
rnn5.add(LeakyReLU())
rnn5.add(Dense(units=12, activation='linear'))
rnn5.add(LeakyReLU())
rnn5.add(Dense(units=1, activation='linear'))
rnn5.add(LeakyReLU())
rnn5.compile(optimizer='adam', loss='mean_squared_error')
rnn5.fit(X5, y5, epochs=20, batch_size=100)

In [ ]:
rnn6 = Sequential()
rnn6.add(LSTM(units=100, activation='linear', input_shape=(None, 3)))
rnn6.add(LeakyReLU())
rnn6.add(Dense(units=50, activation='linear'))
rnn6.add(LeakyReLU())
rnn6.add(Dense(units=25, activation='linear'))
rnn6.add(LeakyReLU())
rnn6.add(Dense(units=12, activation='linear'))
rnn6.add(LeakyReLU())
rnn6.add(Dense(units=1, activation='linear'))
rnn6.add(LeakyReLU())
rnn6.compile(optimizer='adam', loss='mean_squared_error')
rnn6.fit(X6, y6, epochs=20, batch_size=100)

In [ ]:
rnn7 = Sequential()
rnn7.add(LSTM(units=100, activation='linear', input_shape=(None, 3)))
rnn7.add(LeakyReLU())
rnn7.add(Dense(units=50, activation='linear'))
rnn7.add(LeakyReLU())
rnn7.add(Dense(units=25, activation='linear'))
rnn7.add(LeakyReLU())
rnn7.add(Dense(units=12, activation='linear'))
rnn7.add(LeakyReLU())
rnn7.add(Dense(units=1, activation='linear'))
rnn7.add(LeakyReLU())
rnn7.compile(optimizer='adam', loss='mean_squared_error')
rnn7.fit(X7, y7, epochs=20, batch_size=100)

In [ ]:
rnn8 = Sequential()
rnn8.add(LSTM(units=100, activation='linear', input_shape=(None, 3)))
rnn8.add(LeakyReLU())
rnn8.add(Dense(units=50, activation='linear'))
rnn8.add(LeakyReLU())
rnn8.add(Dense(units=25, activation='linear'))
rnn8.add(LeakyReLU())
rnn8.add(Dense(units=12, activation='linear'))
rnn8.add(LeakyReLU())
rnn8.add(Dense(units=1, activation='linear'))
rnn8.add(LeakyReLU())
rnn8.compile(optimizer='adam', loss='mean_squared_error')
rnn8.fit(X8, y8, epochs=20, batch_size=100)

In [ ]:
rnn9 = Sequential()
rnn9.add(LSTM(units=100, activation='linear', input_shape=(None, 3)))
rnn9.add(LeakyReLU())
rnn9.add(Dense(units=50, activation='linear'))
rnn9.add(LeakyReLU())
rnn9.add(Dense(units=25, activation='linear'))
rnn9.add(LeakyReLU())
rnn9.add(Dense(units=12, activation='linear'))
rnn9.add(LeakyReLU())
rnn9.add(Dense(units=1, activation='linear'))
rnn9.add(LeakyReLU())
rnn9.compile(optimizer='adam', loss='mean_squared_error')
rnn9.fit(X9, y9, epochs=20, batch_size=100)

In [ ]:
rnn10 = Sequential()
rnn10.add(LSTM(units=100, activation='linear', input_shape=(None, 3)))
rnn10.add(LeakyReLU())
rnn10.add(Dense(units=50, activation='linear'))
rnn10.add(LeakyReLU())
rnn10.add(Dense(units=25, activation='linear'))
rnn10.add(LeakyReLU())
rnn10.add(Dense(units=12, activation='linear'))
rnn10.add(LeakyReLU())
rnn10.add(Dense(units=1, activation='linear'))
rnn10.add(LeakyReLU())
rnn10.compile(optimizer='adam', loss='mean_squared_error')
rnn10.fit(X10, y10, epochs=20, batch_size=100)

In [ ]:
rnn11 = Sequential()
rnn11.add(LSTM(units=100, activation='linear', input_shape=(None, 3)))
rnn11.add(LeakyReLU())
rnn11.add(Dense(units=50, activation='linear'))
rnn11.add(LeakyReLU())
rnn11.add(Dense(units=25, activation='linear'))
rnn11.add(LeakyReLU())
rnn11.add(Dense(units=12, activation='linear'))
rnn11.add(LeakyReLU())
rnn11.add(Dense(units=1, activation='linear'))
rnn11.add(LeakyReLU())
rnn11.compile(optimizer='adam', loss='mean_squared_error')
rnn11.fit(X11, y11, epochs=20, batch_size=100)

In [ ]:
# making predictions for channel 1 and channel 2
pred_rnn1 = rnn1.predict(test1)
pred_rnn2 = rnn2.predict(test2)
pred_rnn3 = rnn1.predict(test3)
pred_rnn4 = rnn4.predict(test4)
pred_rnn5 = rnn5.predict(test5)
pred_rnn6 = rnn6.predict(test6)
pred_rnn7 = rnn7.predict(test7)
pred_rnn8 = rnn8.predict(test8)
pred_rnn9 = rnn9.predict(test9)
pred_rnn10 = rnn10.predict(test10)
pred_rnn11 = rnn11.predict(test11)

In [ ]:
# saving the LSTM predicitons in wav format
write('pred_rnn.wav', rate, pd.concat([pd.DataFrame(pred_rnn1.astype('int16')), pd.DataFrame(pred_rnn2.astype('int16'))], axis=1).values)
# saving the original music in wav format
write('original.wav',rate, pd.concat([music1.iloc[160001 : 400000, :], music2.iloc[160001 : 400000, :]], axis=0).values)

In [ ]:
!pip install gtts

In [ ]:
from gtts import gTTS
from IPython.display import Audio

sound_file = 'pred_rnn.wav'
Audio(sound_file, autoplay=True)